In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile7 = pd.read_pickle("BBB_Train_Decile7.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile7.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile7, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 7')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 5s 2ms/step - loss: 0.3580 - acc: 0.8727 - val_loss: 0.3304 - val_acc: 0.8769
Epoch 36/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3581 - acc: 0.8743 - val_loss: 0.3309 - val_acc: 0.8756
Epoch 37/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3547 - acc: 0.8714 - val_loss: 0.3299 - val_acc: 0.8782
Epoch 38/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3541 - acc: 0.8740 - val_loss: 0.3275 - val_acc: 0.8795
Epoch 39/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3603 - acc: 0.8717 - val_loss: 0.3292 - val_acc: 0.8795
Epoch 40/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3610 - acc: 0.8675 - val_loss: 0.3254 - val_acc: 0.8782
Epoch 41/75
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3516 - acc: 0.8763 - val_loss: 0.3246 - val_acc: 0.8795
Epoch 42/75
3087/3087 [==============================] - 5s 2ms/ste

Epoch 20/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3196 - acc: 0.8792 - val_loss: 0.3015 - val_acc: 0.8795
Epoch 21/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3151 - acc: 0.8763 - val_loss: 0.3013 - val_acc: 0.8821
Epoch 22/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3168 - acc: 0.8814 - val_loss: 0.2987 - val_acc: 0.8808
Epoch 23/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3141 - acc: 0.8763 - val_loss: 0.3049 - val_acc: 0.8834

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 24/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3144 - acc: 0.8798 - val_loss: 0.3003 - val_acc: 0.8808
Epoch 25/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3082 - acc: 0.8837 - val_loss: 0.2994 - val_acc: 0.8808
Epoch 26/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3136 - acc: 0.8779 - val_loss

3087/3087 [==============================] - 6s 2ms/step - loss: 0.4681 - acc: 0.8008 - val_loss: 0.4283 - val_acc: 0.8290
Epoch 4/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4391 - acc: 0.8183 - val_loss: 0.3982 - val_acc: 0.8420
Epoch 5/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4053 - acc: 0.8448 - val_loss: 0.3830 - val_acc: 0.8497
Epoch 6/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3947 - acc: 0.8503 - val_loss: 0.3628 - val_acc: 0.8588
Epoch 7/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3806 - acc: 0.8614 - val_loss: 0.3475 - val_acc: 0.8705
Epoch 8/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3782 - acc: 0.8630 - val_loss: 0.3371 - val_acc: 0.8744
Epoch 9/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3683 - acc: 0.8691 - val_loss: 0.3285 - val_acc: 0.8769
Epoch 10/100
3087/3087 [==============================] - 6s 2ms/st

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3335 - acc: 0.8698 - val_loss: 0.3088 - val_acc: 0.8769
Epoch 63/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3361 - acc: 0.8691 - val_loss: 0.3085 - val_acc: 0.8769
Epoch 64/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3315 - acc: 0.8733 - val_loss: 0.3089 - val_acc: 0.8769
Epoch 65/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3241 - acc: 0.8720 - val_loss: 0.3088 - val_acc: 0.8769
Epoch 66/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3286 - acc: 0.8720 - val_loss: 0.3088 - val_acc: 0.8769
Epoch 67/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3354 - acc: 0.8753 - val_loss: 0.3085 - val_acc: 0.8769
Epoch 68/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3370 - acc: 0.8733 - val_loss: 0.3085 - val_acc: 0.8769
Epoch 69/100
3087/3087 [==============================] - 6s 

...

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3922 - acc: 0.8617 - val_loss: 0.3629 - val_acc: 0.8640
Epoch 27/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3925 - acc: 0.8659 - val_loss: 0.3599 - val_acc: 0.8731
Epoch 28/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3861 - acc: 0.8701 - val_loss: 0.3564 - val_acc: 0.8795
Epoch 29/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3838 - acc: 0.8682 - val_loss: 0.3548 - val_acc: 0.8756
Epoch 30/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3828 - acc: 0.8672 - val_loss: 0.3527 - val_acc: 0.8808
Epoch 31/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3857 - acc: 0.8682 - val_loss: 0.3524 - val_acc: 0.8744
Epoch 32/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3864 - acc: 0.8656 - val_loss: 0.3504 - val_acc: 0.8808
Epoch 33/125
3087/3087 [==============================] - 6s 

Epoch 86/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3463 - acc: 0.8788 - val_loss: 0.3278 - val_acc: 0.8756
Epoch 87/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3593 - acc: 0.8763 - val_loss: 0.3277 - val_acc: 0.8756
Epoch 88/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3515 - acc: 0.8776 - val_loss: 0.3280 - val_acc: 0.8756
Epoch 89/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3526 - acc: 0.8766 - val_loss: 0.3282 - val_acc: 0.8756
Epoch 90/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3523 - acc: 0.8750 - val_loss: 0.3277 - val_acc: 0.8756
Epoch 91/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3539 - acc: 0.8746 - val_loss: 0.3281 - val_acc: 0.8756
Epoch 92/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3576 - acc: 0.8730 - val_loss: 0.3279 - val_acc: 0.8769
Epoch 93/125
3087/3087 [========================

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3369 - acc: 0.8717 - val_loss: 0.3028 - val_acc: 0.8808
Epoch 21/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3255 - acc: 0.8733 - val_loss: 0.3126 - val_acc: 0.8782
Epoch 22/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3224 - acc: 0.8743 - val_loss: 0.2976 - val_acc: 0.8821
Epoch 23/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3308 - acc: 0.8714 - val_loss: 0.2982 - val_acc: 0.8808
Epoch 24/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3255 - acc: 0.8727 - val_loss: 0.2985 - val_acc: 0.8795
Epoch 25/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3270 - acc: 0.8730 - val_loss: 0.2990 - val_acc: 0.8782
Epoch 26/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3336 - acc: 0.8737 - val_loss: 0.2982 - val_acc: 0.8834
Epoch 27/100
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3183 - acc: 0.8766 - val_loss: 0.2970 - val_acc: 0.8795
Epoch 79/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3162 - acc: 0.8798 - val_loss: 0.2967 - val_acc: 0.8808
Epoch 80/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3109 - acc: 0.8801 - val_loss: 0.2967 - val_acc: 0.8808
Epoch 81/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3168 - acc: 0.8785 - val_loss: 0.2968 - val_acc: 0.8795
Epoch 82/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3183 - acc: 0.8782 - val_loss: 0.2968 - val_acc: 0.8795
Epoch 83/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3173 - acc: 0.8814 - val_loss: 0.2967 - val_acc: 0.8795
Epoch 84/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3110 - acc: 0.8834 - val_loss: 0.2966 - val_acc: 0.8808
Epoch 85/100
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3368 - acc: 0.8685 - val_loss: 0.2996 - val_acc: 0.8718
Epoch 38/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3236 - acc: 0.8753 - val_loss: 0.3004 - val_acc: 0.8756
Epoch 39/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3337 - acc: 0.8733 - val_loss: 0.2996 - val_acc: 0.8731
Epoch 40/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3187 - acc: 0.8788 - val_loss: 0.3007 - val_acc: 0.8756

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 41/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3259 - acc: 0.8792 - val_loss: 0.3002 - val_acc: 0.8744
Epoch 42/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3265 - acc: 0.8782 - val_loss: 0.3006 - val_acc: 0.8769
Epoch 43/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3242 - acc: 0.8733 - val_loss: 0.3001 - val_acc: 0.8756
Ep

In [ ]:
0.8297